# 💡 MechOnFluid 本体加载与推理演示手册

> 本文档旨在系统性展示如何基于 MechOn-Fluid 本体进行 **OWL 本体的加载、推理与可视化分析**。  
> 适用于教学演示、模型验证与垂直领域知识工程的入门实践。

你将学习以下内容：
1. 使用owlready2加载聚合文件-创建world
2. 本体-图谱输出 使用networkx
3. networkx应用
4. (Bonus)Toward GraphRAG

---

🧑‍💻 文档作者：庞嘉顺  
📅 编写日期：2025 年 8 月  
📘 项目背景：本 notebook 属于 [OntoPilot] 与 [SessionSync] 项目的模块示范之一，以力学本体 [MechOn] 作为数据集，从零实现LLM逻辑松耦合附件的功能。

本体内容：MechOn-Fluid

<div align="center">
    <img src="../docs/sessync.png" width="700" alt="sessync logo" />
</div>

## 1 · Owlready2 — `World`

> **`World` = RDF/OWL 的虚拟环境**  
> 通过隔离图（Graph）来实现 **聚合 · 隔离 · 持久化** 三大功能。

| 功能 | 作用 |
| --- | --- |
| **聚合 (Aggregation)** | 把若干本体加载到同一 `World` 👉 跨本体推理 / 查询一致 |
| **隔离 (Isolation)** | 多个 `World` 彼此独立 👉 版本对比、单元测试、多租户 |
| **持久化 (Persistence)** | `world.set_backend()` 写入 SQLite/MySQL 👉 秒级冷启动 |
---

### 1.1 · 聚合示例
```python
world = World()
# ① 先进行IRI与本地路径的映射，再导入
phy = world.get_ontology("https://purl.mechon.org/ont/phy/0.1.0").load()
mat = world.get_ontology("https://purl.mechon.org/ont/mat/0.1.0").load()

# #(可选) - 先进行IRI与本地路径的映射，再导入
# phy = world.get_ontology("https://purl.mechon.org/ont/phy/0.1.0") \
#            .load(file = "./phy.core.ttl")

# mat = world.get_ontology("https://purl.mechon.org/ont/mat/0.1.0") \
#            .load(file = "./mat.core.ttl")

# ② 现在两份本体都在同一个 World 里，可以正常跨本体引用
print("Classes in phy:", len(list(phy.classes())))
print("Classes in mat:", len(list(mat.classes())))
```
>以上两份本体位于同一 world，可以直接互相引用、推理。
---


### 1.2 · 隔离示例
```python
from owlready2 import World

# 独立工作区 A
world_a = World()
onto_a  = world_a.get_ontology("file://A.owl").load()

# 独立工作区 B
world_b = World()
onto_b  = world_b.get_ontology("file://B.owl").load()

```
> 典型场景：版本回归测试、用户级隔离、并行实验等。
---

### 1.3 · 储存示例
```python
world = World()
w.set_backend("mechon.sqlite3")
```
> 支持 SQLite / MySQL / PostgreSQL 等。首次 load() 解析后会写入 DB，后续启动即可秒级恢复。
---

### 1.4 · 何时需要新建World?
| 直接用 default_world | 新建独立 World() |
| --- | --- |
| **单一项目、所有本体天然耦合** | 多版本 / 多租户 / 批量测试 |
| **Quick & Dirty 脚本原型** | 需要持久化、大规模生产服务 |
| **性能开销可忽略** | 希望冷启动加速或彻底隔离 |

# Practice 1 导入MechOn-Fluid本体集到world
- 导出本体集
- 导入桥接文件
- 推理验证

In [18]:
from owlready2 import *
from pathlib import Path
from rdflib import Graph, OWL, RDF
from rdflib.namespace import RDFS, Namespace
from collections import Counter
import textwrap
import tempfile

AggregatedUrl = "./examples/2/mechon-fluid-bridge.owl"

# 定义 URI → 本地文件路径的映射关系（模拟 catalog.xml 的功能）
uri_map = {
    "https://purl.mechon.org/ont/calc/0.1.0": "./examples/2/calc.core.ttl",
    "https://purl.mechon.org/ont/mat/0.1.0": "./examples/2/mat.core.ttl",
    "https://purl.mechon.org/ont/phy/0.1.0": "./examples/2/phy.core.ttl",
    "https://purl.mechon.org/ont/geo/0.1.0": "./examples/2/geo.core.ttl",
    "https://purl.mechon.org/ont/bridge/qk-alignment/0.1.0": "./examples/2/qk-alignment.ttl",
    "https://purl.mechon.org/ont/bridge/qk-bridge/0.1.0": "./examples/2/qudt-clip/qk-bridge.ttl",
}

# ---------- 主流程 ----------

def load_with_imports(graph, path_or_uri, uri_map, loaded=set(), loaded_files=[]):
    """递归加载某本体及其所有 owl:imports；返回已加载文件列表"""
    graph.parse(path_or_uri, format="turtle")
    loaded_files.append(path_or_uri)

    for _, _, o in graph.triples((None, OWL.imports, None)):
        uri = str(o)
        if uri in loaded:
            continue
        if uri in uri_map:
            print(f"🔁  导入 {uri}")
            loaded.add(uri)
            load_with_imports(graph, uri_map[uri], uri_map, loaded, loaded_files)
        else:
            print(f"⚠️  未知 import URI: {uri}（未加载）")
    return loaded_files

g = Graph()
loaded_files = load_with_imports(g, AggregatedUrl, uri_map)

tmp_xml = tempfile.mkstemp(suffix=".owl")[1]
g.serialize(tmp_xml, format="xml")

world_a = World()

onto = world_a.get_ontology(tmp_xml).load()

# ------- 验证 ---------
print(">>> Ontology object:", onto)
print("    • base IRI   :", onto.base_iri)
print("    • loaded from:", onto.loaded_from)         # 应显示 file:///…/mechon-fluid.owl

print("\n>>> World statistics")
print("    • triples    :", len(world_a.as_rdflib_graph()))
print("    • classes    :", len(list(world_a.classes())))
print("    • individuals:", len(list(world_a.individuals())))

sync_reasoner(world_a)


🔁  导入 https://purl.mechon.org/ont/calc/0.1.0
🔁  导入 https://purl.mechon.org/ont/geo/0.1.0
🔁  导入 https://purl.mechon.org/ont/mat/0.1.0
🔁  导入 https://purl.mechon.org/ont/phy/0.1.0
🔁  导入 https://purl.mechon.org/ont/bridge/qk-bridge/0.1.0
🔁  导入 https://purl.mechon.org/ont/bridge/qk-alignment/0.1.0
>>> Ontology object: get_ontology("https://purl.mechon.org/ont/bridge/qk-bridge/")
    • base IRI   : https://purl.mechon.org/ont/bridge/qk-bridge/
    • loaded from: None

>>> World statistics
    • triples    : 29022
    • classes    : 73
    • individuals: 10


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp D:\miniconda3\envs\MechOn\Lib\site-packages\owlready2\hermit;D:\miniconda3\envs\MechOn\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/41888/AppData/Local/Temp/tmpueouma2x
* Owlready2 * HermiT took 2.5804636478424072 seconds
* Owlready * Equivalenting: mat.MatrixSymType mat.ZeroOrderTensor
* Owlready * Equivalenting: mat.ZeroOrderTensor mat.MatrixSymType
* Owlready * Equivalenting: mat.StrainRateTensor mat.SymmetricTensor
* Owlready * Equivalenting: mat.SymmetricTensor mat.StrainRateTensor
* Owlready * Reparenting mat.ThermalConductivity: {mat.ZeroOrderTensor, phy.DerivedQuantity} => {phy.DerivedQuantity, mat.MatrixSymType}
* Owlready * Reparenting mat.InternalEnergy: {mat.ZeroOrderTensor, phy.DerivedQuantity} => {phy.DerivedQuantity, mat.MatrixSymType}
* Owlready * Reparenting mat.Compressibility: {mat.ZeroOrderTensor, phy.DerivedQuantity} => {phy.DerivedQuantity, m

# 使用 **networkx** 承接 Owlready2-World 中提取的 OWL DL 本体  
*—— 面向知识图谱分析与原型算法的快速桥梁 ——*

## 1. 场景概述
在许多语义应用（GraphRAG、合规校核、知识发现等）中，我们常常先用 **Owlready2** 将多个 OWL DL 本体加载到 `World`，并完成推理与一致性检验。接下来若想做**图算法**（最短路径、社区检测、中心性分析……），直接在 RDF 层面实现既繁琐又低效。  
**networkx** 提供了轻量级的 Python 图操作接口，可把 `World` 中的本体实例转成“属性图”，在纯内存里快速迭代算法原型。


---

## 2. 为什么选择 **networkx**

| 需求 | networkx 优势 | 与 Owlready2 配合点 |
|------|---------------|--------------------|
| **零依赖** | 纯 Python，pip 即用 | 与 Owlready2 同生态，不需额外后端 |
| **算法丰富** | 内置图算法 | 直接复用推理后得到的关系 |
| **原型敏捷** | 万级节点秒级执行 | 与 `World` 的“虚拟环境”理念一致 |
| **数据互通** | 可导出为 JSON、GEXF、Pandas | 方便后续导入 Neo4j / 视图层 |

---

## 3. 基础流程

### 3.1 从 `World` 提取三元组
```python
from owlready2 import World

world = World(filename="kg.sqlite3")  # 持久化加载
onto  = world.get_ontology("https://purl.mechon.org/ont/mat#").load()
world.sync_reasoner()                 # 先做 DL 推理

triples = list(world.as_rdflib_graph())  # 转成 rdflib.Graph
```

### 3.2 构造NetworkX属性图
```python
import networkx as nx
from rdflib.namespace import RDF, OWL

G = nx.MultiDiGraph()      # 保留边向与多重关系
for s, p, o in triples:
    # 只取 Individual → Individual/ObjectProperty
    if (p != RDF.type) and (not isinstance(o, (str, int, float))):
        G.add_edge(s, o, label=p)     # 节点即 URIRef, 边属性存谓词

# 可选：过滤 Class 层，只保留 ABox
abox_nodes = [n for n in G if (n, RDF.type, OWL.Class) not in triples]
G = G.subgraph(abox_nodes).copy()

```

### 3.3 运行示例算法
```python
# 社区检测（Louvain）
from networkx.algorithms.community import louvain_communities
comms = louvain_communities(G.to_undirected())

# 节点中心性
pr = nx.pagerank(G, alpha=0.85)
```
---

## 4. 常见用例
| 用例 | 说明 | 代码参考 |
|------|---------------|--------------------|
| **子图分割 (意图切片)** | 根据用户查询把相关节点做 BFS k-hop 子图；减少后续 LLM Prompt 体积 | 留 |
| **社区摘要** | 对 Louvain 划分的社区提摘要，再喂给 LLM 做章节标题 | 留 |
| **推理结果验证** | 比较推理前后边集差异，检测不一致或冗余边	 | 留 |
| **可视化** | 使用`pyvis`, `cytoscape.js`快速绘图 | 留 |

networkx在应用中可以被neo4j丝滑替代